## Transfer Learning

> Transfer Learning use when we dun have high computational power and we train our model with our custom datasets. because we have already pre-trained models which is developed by researchers for competitions.. so we use and utilies that models and edit with our own requirements and train
* **Large** and **different** dataset ---> Train the hole model
* **large** and **Similar** dataset ---> Fine Tuning
* **Small** and **different** dataset ---> Fine tuning
* **Small** and **Similar** dataset ---> Transfer Learning

### Download the dataset

In [ ]:
!pwd
!cd /content/drive/MyDrive

/content


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/drive/MyDrive/cats_and_dogs_filtered.zip

--2021-03-27 10:09:40--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 172.217.12.240, 142.250.73.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.9.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/content/drive/MyDrive/cats_and_dogs_filtered.zip’

/content/drive/MyDr 100%[===================>]  65.43M  71.5MB/s    in 0.9s    

2021-03-27 10:09:41 (71.5 MB/s) - ‘/content/drive/MyDrive/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
!pip install tqdm

### Data Preprocessing

##### Import project dependencies

In [56]:
import tensorflow as tf
import numpy as np
import os
import zipfile
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

% matplotlib inline
tf.__version__

'2.4.1'

### Unzip the dataset

In [57]:
zip_filder_path = '/content/drive/MyDrive/cats_and_dogs_filtered.zip'

In [58]:
zip_file = zipfile.ZipFile(file=zip_filder_path, mode='r')

In [59]:
zip_file.extractall('/content/')

In [60]:
zip_file.close()

In [61]:
# or you can unzip the dataset by executing the below command.. Note: this works just in colab

# !unzip '/content/drive/MyDrive/cats_and_dogs_filtered.zip' -d '/content/'

### Setting up dataset path

In [62]:
dataset_path = '/content/cats_and_dogs_filtered'

In [63]:
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

### building the pre-trained model
#### Load the pre-trained model(MobileNet V2)

1. let's define shape for our images 128x128
2. load the model from keras..
3. mobilenet have no doc-string so we dun know which arguments/parameter to pass. you can check in google, here we are passing the important ones.
    * `input_shape` pass the dimensions of our images. we store dimensions of our images in IMG_shape variable
    * `include_top` mobilenet model is trained on imagenet dataset and the ouput layers is design for imagenet dataset. which have so many classes. so we remove the output/top/head layer so we can pass our own classes
    * `weights` we define the name of the dataset on which the pre-trained model is trained. in our case we pass `imagenet`

In [64]:
IMG_Shape = (128, 128, 3)

In [65]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_Shape, include_top=False, weights='imagenet')

In [66]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

### Freezing the base_model
  * `Freez_model` when we are using pre-trained model we will remove some of the layers. that we define in loading time.. so we will freez all the layers inside trained model network. and will trained only those layers which we are adding to the model.

In [67]:
base_model.trainable = False

### Defining the custom head/out_put layer for our network
  * we load our model and removed the head/output_layer.
  * now we are adding the top layer/head/output_layer according to our requirments

In [68]:
base_model.output, base_model.output.shape

(<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>,
 TensorShape([None, 4, 4, 1280]))

* you guys can see that the shape is different. \
two techniques we have
1. Flatten layer
2. global average pooling layer (global average pooling layer will take all the pixels and will get the average of that pixels)

<img src='https://i.stack.imgur.com/wCL4l.png'>

In [69]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [70]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_2')>

* we have changed the x and y values of the base_model which was 4x4 now we set to nan
* now we have to add the output layer or prediction layer

In [71]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

* we specify units/neurons to two. because we have two classes (cats and dogs)


### Defining a model

In [72]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [73]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

### Compiling the Model
* Optimizer: RMSprop (lr=0.0001)
* loss: binary_crossentropy
* matrics: accuracy

1. rmsprop is proven the best optimizer for mobilenet
2. we are using pre-trained model so we will specify very low learning_rate

In [74]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

### Creating Data Generators / Normalization

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [75]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [76]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [77]:
valid_generator = data_gen_validation.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 1000 images belonging to 2 classes.


## Training the Model

In [80]:
model.fit_generator(train_generator, epochs=20, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
16/16 [==============================] - 10s 611ms/step - loss: 0.1912 - accuracy: 0.9300 - val_loss: 0.1762 - val_accuracy: 0.9440
Epoch 2/20
16/16 [==============================] - 9s 598ms/step - loss: 0.1822 - accuracy: 0.9330 - val_loss: 0.1681 - val_accuracy: 0.9460
Epoch 3/20
16/16 [==============================] - 9s 586ms/step - loss: 0.1744 - accuracy: 0.9355 - val_loss: 0.1612 - val_accuracy: 0.9520
Epoch 4/20
16/16 [==============================] - 9s 584ms/step - loss: 0.1673 - accuracy: 0.9375 - val_loss: 0.1549 - val_accuracy: 0.9530
Epoch 5/20
16/16 [==============================] - 9s 585ms/step - loss: 0.1612 - accuracy: 0.9400 - val_loss: 0.1492 - val_accuracy: 0.9550
Epoch 6/20
16/16 [==============================] - 9s 585ms/step - loss: 0.1552 - accuracy: 0.9420 - val_loss: 0.1439 - val_accuracy: 0.9540
Epoch 7/20
16/16 [==============================] - 9s 591ms/step - loss: 0.1499 - accuracy: 0.9420 - val_loss: 0.1392 - val_accuracy: 0.9550
Epoch

### Training Evaluation Model

In [89]:
test_loss, test_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [91]:
test_accuracy

0.9670000076293945

In [94]:
print(f'Accuracy after Transfer Learning {test_accuracy: .2f}%')

Accuracy after Transfer Learning  0.97%


> The Accuracy is quite amazing we got 95% accuracy on train data and 97% on test data. \
Now we are moving forward and imporoving the accuracy  by `Fine Tuning`
### Fine Tuning



There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [96]:
base_model.trainable = True

In [97]:
print(f'Number of layers in pre-trained model MobileNet V2 {len(base_model.layers)}')

Number of layers in pre-trained model MobileNet V2 154


> as you can see that mobilenet have 154 layers and we are not going to train overall network. because this well take much time and the gradient will be much different. \
So are going to train some layers of the network. from `100` to the last(154) layer

In [98]:
fine_tune_at = 100

> we unfrozed the whole network earlier. so Now we will freez the network again before 100th layer

### Freezing first layers from 100th

In [99]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Compiling the model for fine-tuning

In [100]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [101]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 16s 680ms/step - loss: 0.2630 - accuracy: 0.8812 - val_loss: 0.0701 - val_accuracy: 0.9710
Epoch 2/5
16/16 [==============================] - 10s 601ms/step - loss: 0.0240 - accuracy: 0.9979 - val_loss: 0.0693 - val_accuracy: 0.9750
Epoch 3/5
16/16 [==============================] - 10s 608ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0858 - val_accuracy: 0.9700
Epoch 4/5
16/16 [==============================] - 10s 605ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0904 - val_accuracy: 0.9700
Epoch 5/5
16/16 [==============================] - 10s 603ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0973 - val_accuracy: 0.9700


### Evaluating the Fine-tuned model

In [102]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [103]:
print(f'Testing Accuracy after Fine-Tuning : {valid_accuracy: .2f}')

Testing Accuracy after Fine-Tuning :  0.97
